# OpenAI API 고급 활용

## 목차
1. 프롬프트 엔지니어링 (OpenAI 공식 6가지 전략)
2. 고급 프롬프트 기법 (CoT, Few-shot, Auto-CoT 등)
3. Function Calling
4. RAG (Retrieval-Augmented Generation)

---

In [2]:
# 환경 설정
import os
import json
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import tiktoken

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✅ 환경 설정 완료")

✅ 환경 설정 완료


# 1. 프롬프트 엔지니어링 - OpenAI 공식 6가지 전략

출처: [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)

## 전략 1: Write Clear Instructions (명확한 지시 작성)

### 1.1 세부 사항 포함하기

In [5]:
def compare_prompts(bad_prompt, good_prompt):
    """두 프롬프트 비교"""
    print("❌ 나쁜 예시:")
    print(f"Prompt: {bad_prompt}\n")
    
    response1 = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": bad_prompt}],
        temperature=0.7
    )
    print(f"Response: {response1.choices[0].message.content}\n")
    
    print("=" * 80)
    
    print("\n✅ 좋은 예시:")
    print(f"Prompt: {good_prompt}\n")
    
    response2 = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": good_prompt}],
        temperature=0.7
    )
    print(f"Response: {response2.choices[0].message.content}")

# 예시
compare_prompts(
    bad_prompt="파이썬 피보나치 만들어줘",
    good_prompt="""파이썬으로 피보나치 수열을 계산하는 함수를 만들어주세요.
    
    요구사항:
    - 함수명: fibonacci
    - 입력: 정수 n (n번째 피보나치 수)
    - 출력: n번째 피보나치 수
    - 재귀 방식으로 구현
    - docstring과 타입 힌트 포함
    - 예제 사용법 3개 포함
    - 다른 말들 제외하고 딱 함수만 생성해줘
    """
)

❌ 나쁜 예시:
Prompt: 파이썬 피보나치 만들어줘

Response: 파이썬에서 피보나치 수열을 생성하는 간단한 함수를 작성할 수 있습니다. 피보나치 수열은 첫 두 항이 0과 1이며, 그 이후의 항은 앞의 두 항의 합으로 정의됩니다. 아래는 피보나치 수열을 생성하는 몇 가지 방법입니다.

### 1. 재귀를 이용한 방법

```python
def fibonacci_recursive(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci_recursive(n - 1) + fibonacci_recursive(n - 2)

# 예시: 첫 10개의 피보나치 수 출력
for i in range(10):
    print(fibonacci_recursive(i))
```

### 2. 반복문을 이용한 방법

```python
def fibonacci_iterative(n):
    a, b = 0, 1
    for _ in range(n):
        print(a, end=" ")
        a, b = b, a + b

# 예시: 첫 10개의 피보나치 수 출력
fibonacci_iterative(10)
```

### 3. 리스트를 이용한 방법

```python
def fibonacci_list(n):
    fib_sequence = []
    a, b = 0, 1
    for _ in range(n):
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

# 예시: 첫 10개의 피보나치 수 출력
print(fibonacci_list(10))
```

원하는 방식에 따라 위의 코드 중 하나를 선택하여 사용하면 됩니다. 각 방법은 피보나치 수열의 n번째 항을 출력하거나 리스트로 반환하는 기능을 가지고 있습니다.


✅ 좋은 예시:

### 1.2 페르소나 지정하기

In [6]:
# 페르소나에 따른 답변 차이
question = "양자 컴퓨팅을 설명해줘"

personas = [
    "당신은 5살 어린이에게 설명하는 선생님입니다.",
    "당신은 대학 물리학과 교수입니다.",
    "당신은 기술 블로그 작가로, 일반인이 이해하기 쉽게 설명합니다."
]

for i, persona in enumerate(personas, 1):
    print(f"\n{'='*80}")
    print(f"페르소나 {i}: {persona}")
    print(f"{'='*80}\n")
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": persona},
            {"role": "user", "content": question}
        ],
        temperature=0.7,
        max_tokens=200
    )
    print(response.choices[0].message.content)


페르소나 1: 당신은 5살 어린이에게 설명하는 선생님입니다.

안녕! 양자 컴퓨팅에 대해 쉽게 설명해줄게.

컴퓨터는 정보를 처리하는 도구야. 우리가 아는 컴퓨터는 '비트'라는 아주 작은 조각을 사용해 정보를 저장하고 처리해. 비트는 0 아니면 1로만 표현할 수 있어. 예를 들어, 불이 꺼져 있으면 0, 켜져 있으면 1이라고 생각할 수 있어.

하지만 양자 컴퓨터는 '큐비트'라는 특별한 조각을 사용해. 큐비트는 0이나 1일 수도 있지만, 동시에 0이면서 1일 수도 있어! 이걸 '중첩'이라고 해. 그래서 양자 컴퓨터는 훨씬 더 많은 정보를 한 번에 처리할 수 있어.

또한, 큐비트는 서로 연결될 수 있는데, 이걸 '얽힘'이라고 해. 이렇게 되면 큐비트들이

페르소나 2: 당신은 대학 물리학과 교수입니다.

양자 컴퓨팅은 양자역학의 원리를 활용하여 정보를 처리하는 컴퓨팅 방식입니다. 전통적인 컴퓨터가 비트(bit)를 사용하여 정보를 0과 1로 표현하는 반면, 양자 컴퓨터는 양자 비트(큐비트, qubit)를 사용합니다. 큐비트는 다음과 같은 특성을 가집니다:

1. **중첩(Superposition)**: 큐비트는 0과 1의 상태를 동시에 가질 수 있습니다. 예를 들어, 하나의 큐비트는 0, 1 또는 이 둘의 조합인 상태를 나타낼 수 있습니다. 이로 인해 양자 컴퓨터는 여러 계산을 동시에 수행할 수 있는 잠재력을 가집니다.

2. **얽힘(Entanglement)**: 두 개 이상의 큐비트가 얽히면, 한 큐비트의 상태가 다른

페르소나 3: 당신은 기술 블로그 작가로, 일반인이 이해하기 쉽게 설명합니다.

양자 컴퓨팅은 전통적인 컴퓨터와는 다른 방식으로 정보를 처리하는 컴퓨터 기술입니다. 이를 이해하기 위해서는 먼저 전통적인 컴퓨터가 어떻게 작동하는지 알아야 합니다.

### 전통적인 컴퓨터
전통적인 컴퓨터는 비트(bit)라는 최소 단위로 정보를 처리합니다. 비트는 0 또는 1의 두 가지 상태만 가질 수 있습니다. 모든 데이터와 프로그램은 이러한 비트의 조합으로 

### 1.3 구분자 사용하기 (Delimiters)

입력과 지시사항을 명확히 구분

In [7]:
text_to_summarize = """
인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습하고 추론하며 
문제를 해결할 수 있도록 하는 기술입니다. 머신러닝은 인공지능의 한 분야로, 
데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 방법입니다. 
딥러닝은 머신러닝의 하위 분야로, 인공신경망을 사용하여 더 복잡한 
패턴을 학습할 수 있습니다.
"""

prompt = f"""
다음 삼중 백틱으로 구분된 텍스트를 한 문장으로 요약하세요.

```
{text_to_summarize}
```

요약:
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)

print("원본 텍스트:")
print(text_to_summarize)
print("\n" + "="*80)
print("\n요약:")
print(response.choices[0].message.content)

원본 텍스트:

인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습하고 추론하며 
문제를 해결할 수 있도록 하는 기술입니다. 머신러닝은 인공지능의 한 분야로, 
데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 방법입니다. 
딥러닝은 머신러닝의 하위 분야로, 인공신경망을 사용하여 더 복잡한 
패턴을 학습할 수 있습니다.



요약:
인공지능은 인간의 지능을 모방하여 학습하고 문제를 해결하는 기술로, 그 중 머신러닝은 데이터에서 패턴을 학습하고, 딥러닝은 인공신경망을 이용해 더 복잡한 패턴을 학습하는 하위 분야입니다.


### 1.4 출력 형식 지정하기

In [8]:
prompt = """
다음 정보를 JSON 형식으로 정리해주세요:

김철수는 서울 출신의 35세 남성으로, 현재 데이터 과학자로 일하고 있습니다. 
그는 Python과 R을 능숙하게 다루며, 취미는 하이킹과 사진촬영입니다.

JSON 형식:
{
  "name": "이름",
  "age": 나이(숫자),
  "gender": "성별",
  "location": "거주지",
  "job": "직업",
  "skills": ["기술1", "기술2"],
  "hobbies": ["취미1", "취미2"]
}
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    response_format={"type": "json_object"},
    temperature=0.3
)

result = json.loads(response.choices[0].message.content)
print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "name": "김철수",
  "age": 35,
  "gender": "남성",
  "location": "서울",
  "job": "데이터 과학자",
  "skills": [
    "Python",
    "R"
  ],
  "hobbies": [
    "하이킹",
    "사진촬영"
  ]
}


### 1.5 조건 충족 확인 요청

In [12]:
prompt = """
다음 텍스트에 지시사항이 포함되어 있으면 그대로 따르고,
포함되어 있지 않으면 "지시사항이 없습니다"라고 답하세요.

텍스트: 

차를 만드는 방법: 1) 물을 끓인다. 2) 찻잎을 넣는다. 3) 3분 우린다.

지시사항을 단계별로 다음 형식으로 다시 작성하세요:
Step 1 - ...
Step 2 - ...
...
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)

print(response.choices[0].message.content)

print("\n" + "="*80 + "\n")

# 지시사항이 없는 경우
prompt2 = """
다음 텍스트에 지시사항이 포함되어 있으면 그대로 따르고,
포함되어 있지 않으면 "지시사항이 없습니다"라고 답하세요.

텍스트: "오늘은 날씨가 좋습니다."
"""

response2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt2}],
    temperature=0.3
)

print(response2.choices[0].message.content)

Step 1 - 물을 끓인다.  
Step 2 - 찻잎을 넣는다.  
Step 3 - 3분 우린다.  


지시사항이 없습니다.


## 전략 2: Provide Reference Text (참조 텍스트 제공)

모델이 정확한 정보를 바탕으로 답변하도록 참조 자료 제공

In [13]:
reference_text = """
회사 휴가 정책:
1. 연차: 입사 1년차 15일, 2년차부터 매년 1일씩 추가 (최대 25일)
2. 병가: 연간 10일, 의사 소견서 필요
3. 경조사: 본인 결혼 5일, 직계가족 사망 3일
4. 휴가 신청: 최소 3일 전 상사 승인 필요
5. 미사용 연차: 다음 해 이월 불가, 금전 보상 가능
"""

prompt = f"""
다음 참조 텍스트를 바탕으로 질문에 답변하세요.
참조 텍스트에 없는 내용은 "참조 자료에서 찾을 수 없습니다"라고 답하세요.

참조 텍스트:
```
{reference_text}
```

질문: 입사 3년차 직원은 연차를 몇 일 받을 수 있나요?
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

print("답변:", response.choices[0].message.content)

# 참조 텍스트에 없는 질문
print("\n" + "="*80 + "\n")

prompt2 = f"""
다음 참조 텍스트를 바탕으로 질문에 답변하세요.
참조 텍스트에 없는 내용은 "참조 자료에서 찾을 수 없습니다"라고 답하세요.

참조 텍스트:
```
{reference_text}
```

질문: 재택근무 정책은 어떻게 되나요?
"""

response2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt2}],
    temperature=0.0
)

print("답변:", response2.choices[0].message.content)

답변: 입사 3년차 직원은 연차를 17일 받을 수 있습니다. (1년차 15일 + 2년차 1일 추가 + 3년차 1일 추가)


답변: 참조 자료에서 찾을 수 없습니다.


## 전략 3: Split Complex Tasks (복잡한 작업 분할)

복잡한 작업을 여러 단순한 하위 작업으로 나누기

In [14]:
# 나쁜 예: 한 번에 너무 많은 것을 요청
bad_prompt = """
다음 코드를 분석하고, 버그를 찾고, 고치고, 최적화하고, 문서화하고, 
테스트 케이스를 작성해주세요:

def calc(a, b, op):
    if op == '+':
        return a + b
    elif op == '-':
        return a - b
    elif op == '*':
        return a * b
    elif op == '/':
        return a / b
"""

# 좋은 예: 단계별로 분할
code = """
def calc(a, b, op):
    if op == '+':
        return a + b
    elif op == '-':
        return a - b
    elif op == '*':
        return a * b
    elif op == '/':
        return a / b
"""

# Step 1: 버그 찾기
step1 = f"""
다음 코드에서 잠재적인 버그나 문제점을 찾아주세요:

```python
{code}
```

버그를 나열하고 각각 설명해주세요.
"""

response1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": step1}],
    temperature=0.3
)

print("Step 1 - 버그 분석:")
print(response1.choices[0].message.content)

# Step 2: 코드 수정
print("\n" + "="*80 + "\n")

step2 = f"""
다음 문제점을 고려하여 코드를 수정해주세요:

문제점:
{response1.choices[0].message.content}

원본 코드:
```python
{code}
```

수정된 코드만 제공하세요.
"""

response2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": step2}],
    temperature=0.3
)

print("Step 2 - 수정된 코드:")
print(response2.choices[0].message.content)

Step 1 - 버그 분석:
주어진 코드에서 잠재적인 버그나 문제점을 다음과 같이 나열할 수 있습니다:

1. **0으로 나누기 오류**:
   - `/` 연산을 수행할 때 `b`가 0인 경우 `ZeroDivisionError`가 발생합니다. 이 경우 코드가 예외를 처리하지 않으므로 프로그램이 중단됩니다.
   - 해결 방법: 나누기 연산을 수행하기 전에 `b`가 0인지 확인하고, 적절한 예외 처리를 추가해야 합니다.

   ```python
   elif op == '/':
       if b == 0:
           raise ValueError("Cannot divide by zero.")
       return a / b
   ```

2. **지원하지 않는 연산자 처리 부족**:
   - `op`가 `+`, `-`, `*`, `/` 중 하나가 아닐 경우, 함수는 아무것도 반환하지 않습니다. 이는 `None`을 반환하게 되어 호출하는 측에서 예기치 않은 동작을 초래할 수 있습니다.
   - 해결 방법: 지원하지 않는 연산자에 대해 적절한 예외를 발생시키거나 기본값을 반환해야 합니다.

   ```python
   else:
       raise ValueError(f"Unsupported operation: {op}")
   ```

3. **입자 유형 검증 부족**:
   - `a`와 `b`가 숫자(정수 또는 부동소수점)인지 확인하지 않습니다. 만약 문자열이나 다른 데이터 유형이 전달되면 TypeError가 발생할 수 있습니다.
   - 해결 방법: 함수 시작 부분에서 `a`와 `b`의 타입을 확인하고, 적절한 예외를 발생시켜야 합니다.

   ```python
   if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
       raise TypeError("Both a and b must be numbers.")
   ```

4. **부동소수점 연산의 정확성 문

## 전략 4: Give Time to "Think" (생각할 시간 주기)

모델이 추론 과정을 단계별로 진행하도록 유도

In [15]:
# 나쁜 예: 바로 답 요구
bad_prompt = """
학생의 수학 풀이가 맞는지 확인해주세요:

문제: x^2 + 5x + 6 = 0의 해는?
학생 답: x = -2 또는 x = 3

정답인가요?
"""

response_bad = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": bad_prompt}],
    temperature=0.3
)

print("❌ 바로 답변 요구:")
print(response_bad.choices[0].message.content)

print("\n" + "="*80 + "\n")

# 좋은 예: 단계별 추론 요청
good_prompt = """
학생의 수학 풀이가 맞는지 단계별로 확인해주세요:

문제: x^2 + 5x + 6 = 0의 해는?
학생 답: x = -2 또는 x = 3

다음 순서로 확인하세요:
1. 먼저 스스로 문제를 풀어보세요
2. 학생의 답과 비교하세요
3. 학생의 답이 맞는지 최종 판단하세요

각 단계를 명확히 보여주세요.
"""

response_good = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": good_prompt}],
    temperature=0.3
)

print("✅ 단계별 추론 요청:")
print(response_good.choices[0].message.content)

❌ 바로 답변 요구:
학생의 답이 틀렸습니다. 주어진 방정식 \(x^2 + 5x + 6 = 0\)을 인수분해하면 다음과 같습니다:

\[
x^2 + 5x + 6 = (x + 2)(x + 3) = 0
\]

따라서, 방정식의 해는 \(x + 2 = 0\) 또는 \(x + 3 = 0\)이므로:

\[
x = -2 \quad \text{또는} \quad x = -3
\]

학생의 답은 \(x = -2\) 또는 \(x = 3\)인데, \(3\)은 잘못된 해입니다. 올바른 해는 \(x = -2\) 또는 \(x = -3\)입니다.


✅ 단계별 추론 요청:
문제: \( x^2 + 5x + 6 = 0 \)의 해를 구해보겠습니다.

### 1단계: 스스로 문제를 풀어보세요

주어진 방정식은 \( x^2 + 5x + 6 = 0 \)입니다. 이 방정식을 인수분해를 통해 풀어보겠습니다.

1. \( x^2 + 5x + 6 \)을 인수분해합니다.
2. 두 수의 곱이 6이고 합이 5인 두 수를 찾습니다. 
   - 2와 3이 해당됩니다.
3. 따라서 인수분해는 다음과 같습니다:
   \[
   (x + 2)(x + 3) = 0
   \]
4. 각 인수에 대해 0이 되는 값을 찾습니다:
   - \( x + 2 = 0 \) → \( x = -2 \)
   - \( x + 3 = 0 \) → \( x = -3 \)

따라서 방정식의 해는 \( x = -2 \) 또는 \( x = -3 \)입니다.

### 2단계: 학생의 답과 비교하세요

학생의 답: \( x = -2 \) 또는 \( x = 3 \)

### 3단계: 학생의 답이 맞는지 최종 판단하세요

학생의 답에서 \( x = -2 \)는 맞지만, \( x = 3 \)는 잘못된 답입니다. 올바른 해는 \( x = -2 \)와 \( x = -3 \)입니다.

결론적으로, 학생의 답은 부분적으로 맞지만, 전체적으로는 틀렸습니다. 올바른 해는 \( x = -2 \)와 \( x = -3 \)입니다.


## 전략 5: Use External Tools (외부 도구 활용)

모델의 약점을 외부 도구로 보완 (다음 섹션에서 Function Calling으로 구현)

In [18]:
# 전략 5: Use External Tools (외부 도구 활용)

# 나쁜 예: 모델에게 직접 복잡한 계산 요청
bad_prompt = "1234567 × 9876543을 계산해주세요."

response_bad = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": bad_prompt}],
    temperature=0.0
)

print("❌ 외부 도구 없이 (모델이 직접 계산):")
print(response_bad.choices[0].message.content)
print()

# 좋은 예: 외부 도구(계산기) 활용
import ast
import operator

def safe_calculate(expression):
    """안전한 계산기"""
    try:
        # 허용된 연산자만 사용
        operators = {
            ast.Add: operator.add,
            ast.Sub: operator.sub,
            ast.Mult: operator.mul,
            ast.Div: operator.truediv,
            ast.Pow: operator.pow,
        }
        
        def eval_node(node):
            if isinstance(node, ast.Num):
                return node.n
            elif isinstance(node, ast.BinOp):
                return operators[type(node.op)](
                    eval_node(node.left), 
                    eval_node(node.right)
                )
            else:
                raise ValueError("지원하지 않는 연산")
        
        tree = ast.parse(expression, mode='eval')
        return eval_node(tree.body)
    except:
        return "계산 오류"

good_prompt = f"""
다음 계산을 하세요:
1234567 × 9876543

계산 과정:
1. 계산식을 정리: "1234567 * 9876543"
2. 계산기 도구를 사용하라고 지시

응답 형식:
계산식: [식]
계산기 사용 필요: Yes
"""

response_good = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": good_prompt}],
    temperature=0.0
)

print("✅ 외부 도구(계산기) 활용:")
print(response_good.choices[0].message.content)

# 실제 계산
result = safe_calculate("1234567 * 9876543")
print(f"\n계산기 결과: {result:,}")

print("\n" + "="*80)
print("\n💡 핵심:")
print("  - 모델이 약한 부분(복잡한 계산, 최신 정보)은 외부 도구 활용")
print("  - 더 정확하고 신뢰할 수 있는 결과")
print("  - Function Calling이 이를 자동화!")

❌ 외부 도구 없이 (모델이 직접 계산):
1234567 × 9876543의 계산 결과는 1219326221입니다.

✅ 외부 도구(계산기) 활용:
계산식: 1234567 × 9876543  
계산기 사용 필요: Yes

계산기 결과: 12,193,254,061,881


💡 핵심:
  - 모델이 약한 부분(복잡한 계산, 최신 정보)은 외부 도구 활용
  - 더 정확하고 신뢰할 수 있는 결과
  - Function Calling이 이를 자동화!


## 전략 6: Test Changes Systematically (체계적 테스트)

프롬프트 변경 효과를 측정

In [19]:
# 다양한 프롬프트 버전 테스트
test_cases = [
    {"input": "파리는 어느 나라 수도?", "expected": "프랑스"},
    {"input": "서울은 어느 나라 수도?", "expected": "대한민국"},
    {"input": "도쿄는 어느 나라 수도?", "expected": "일본"},
]

prompts = [
    "V1 (단순)",
    "V2 (상세)",
]

def test_prompt(version, system_message, test_cases):
    """프롬프트 버전 테스트"""
    print(f"\n테스트: {version}")
    print(f"System: {system_message}")
    print("-" * 80)
    
    correct = 0
    for test in test_cases:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": test["input"]}
            ],
            temperature=0.0,
            max_tokens=50
        )
        answer = response.choices[0].message.content
        is_correct = test["expected"] in answer
        if is_correct:
            correct += 1
        
        print(f"입력: {test['input']}")
        print(f"기대: {test['expected']}")
        print(f"실제: {answer}")
        print(f"결과: {'✅' if is_correct else '❌'}\n")
    
    accuracy = (correct / len(test_cases)) * 100
    print(f"정확도: {accuracy:.1f}% ({correct}/{len(test_cases)})\n")
    return accuracy

# V1 테스트
acc1 = test_prompt(
    "V1 (단순)",
    "질문에 답하세요.",
    test_cases
)

print("="*80)

# V2 테스트
acc2 = test_prompt(
    "V2 (상세)",
    "당신은 지리 전문가입니다. 질문에 정확하고 간결하게 답하세요. 국가 이름만 답변하세요.",
    test_cases
)

print(f"\n📊 결과: V2가 V1보다 {acc2 - acc1:.1f}% 더 정확함")


테스트: V1 (단순)
System: 질문에 답하세요.
--------------------------------------------------------------------------------
입력: 파리는 어느 나라 수도?
기대: 프랑스
실제: 파리는 프랑스의 수도입니다.
결과: ✅

입력: 서울은 어느 나라 수도?
기대: 대한민국
실제: 서울은 대한민국(한국)의 수도입니다.
결과: ✅

입력: 도쿄는 어느 나라 수도?
기대: 일본
실제: 도쿄는 일본의 수도입니다.
결과: ✅

정확도: 100.0% (3/3)


테스트: V2 (상세)
System: 당신은 지리 전문가입니다. 질문에 정확하고 간결하게 답하세요. 국가 이름만 답변하세요.
--------------------------------------------------------------------------------
입력: 파리는 어느 나라 수도?
기대: 프랑스
실제: 프랑스
결과: ✅

입력: 서울은 어느 나라 수도?
기대: 대한민국
실제: 대한민국
결과: ✅

입력: 도쿄는 어느 나라 수도?
기대: 일본
실제: 일본
결과: ✅

정확도: 100.0% (3/3)


📊 결과: V2가 V1보다 0.0% 더 정확함


# 2. 고급 프롬프트 기법

## 2.1 Zero-shot vs Few-shot Learning

In [20]:
# Zero-shot: 예시 없이 바로 작업
zero_shot_prompt = """
다음 문장의 감정을 분류하세요 (긍정/부정/중립):

"이 제품은 가격 대비 성능이 훌륭합니다!"
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": zero_shot_prompt}],
    temperature=0.3
)

print("Zero-shot 결과:")
print(response.choices[0].message.content)

print("\n" + "="*80 + "\n")

# Few-shot: 예시 제공
few_shot_prompt = """
다음 예시를 참고하여 문장의 감정을 분류하세요:

예시 1:
문장: "정말 최고의 경험이었어요!"
감정: 긍정

예시 2:
문장: "너무 실망스러웠습니다."
감정: 부정

예시 3:
문장: "그냥 평범했어요."
감정: 중립

이제 다음 문장을 분류하세요:
문장: "이 제품은 가격 대비 성능이 훌륭합니다!"
감정:
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": few_shot_prompt}],
    temperature=0.3
)

print("Few-shot 결과:")
print(response.choices[0].message.content)

Zero-shot 결과:
이 문장은 긍정적인 감정을 표현하고 있습니다.


Few-shot 결과:
감정: 긍정


## 2.2 Chain of Thought (CoT)

"Let's think step by step" - 단계별 추론을 유도

In [22]:
# CoT 없이
problem = """
철수는 사과 5개를 가지고 있었습니다.
영희에게 2개를 주고, 민수에게서 3개를 받았습니다.
그 다음 날 7개를 더 샀습니다.
철수가 가진 사과는 총 몇 개인가요?
"""

print("❌ CoT 없이:")
response1 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": problem}],
    temperature=0.0
)
print(response1.choices[0].message.content)

print("\n" + "="*80 + "\n")

# CoT 적용
cot_prompt = problem + "\n\n단계별로 생각하며 풀어보세요."

print("✅ CoT 적용:")
response2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": cot_prompt}],
    temperature=0.0
)
print(response2.choices[0].message.content)

❌ CoT 없이:
철수가 가진 사과의 개수는 12개입니다. 처음에 5개를 가졌고, 영희에게 2개를 주었으니 3개가 남았습니다. 그리고 민수에게서 3개를 받아서 총 6개가 되었습니다. 이후 7개를 더 사서 6개에 7개를 더한 총 12개가 됩니다.


✅ CoT 적용:
철수가 가진 사과의 수를 단계별로 계산해 보겠습니다.

1. **처음 사과의 수**: 철수는 처음에 사과 5개를 가지고 있었습니다.
   - 초기 사과 수: 5개

2. **영희에게 사과 주기**: 철수는 영희에게 2개를 주었습니다.
   - 5개 - 2개 = 3개

3. **민수에게서 사과 받기**: 철수는 민수에게서 3개를 받았습니다.
   - 3개 + 3개 = 6개

4. **다음 날 사과 구매**: 철수는 다음 날 7개를 더 샀습니다.
   - 6개 + 7개 = 13개

따라서, 철수가 가진 사과는 총 **13개**입니다.


## 2.3 Few-shot CoT

예시와 함께 추론 과정을 보여주기

In [23]:
few_shot_cot_prompt = """
다음 예시처럼 단계별로 생각하며 문제를 풀어주세요:

예시:
문제: 버스에 23명이 타고 있었습니다. 정류장에서 8명이 내리고 5명이 탔습니다. 버스에 몇 명이 있나요?
풀이:
1. 처음 인원: 23명
2. 8명이 내림: 23 - 8 = 15명
3. 5명이 탐: 15 + 5 = 20명
답: 20명

이제 다음 문제를 풀어주세요:
문제: 가게에 연필이 45자루 있었습니다. 오전에 12자루를 팔고, 오후에 18자루를 새로 입고했습니다. 다음날 15자루를 더 팔았습니다. 남은 연필은 몇 자루인가요?
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": few_shot_cot_prompt}],
    temperature=0.0
)

print(response.choices[0].message.content)

문제: 가게에 연필이 45자루 있었습니다. 오전에 12자루를 팔고, 오후에 18자루를 새로 입고했습니다. 다음날 15자루를 더 팔았습니다. 남은 연필은 몇 자루인가요?

풀이:
1. 처음 연필 수: 45자루
2. 오전에 12자루를 팔음: 45 - 12 = 33자루
3. 오후에 18자루를 입고함: 33 + 18 = 51자루
4. 다음날 15자루를 더 팔음: 51 - 15 = 36자루

답: 36자루


## 2.4 Auto-CoT (자동 Chain of Thought)

모델이 스스로 예시를 생성하고 추론

In [24]:
def auto_cot(question, num_examples=2):
    """Auto-CoT 구현"""
    
    # Step 1: 유사한 예시 생성 요청
    example_prompt = f"""
    다음과 유사한 문제 {num_examples}개와 단계별 풀이를 생성해주세요:
    
    문제 예시: {question}
    
    각 문제마다 명확한 단계별 풀이를 포함하세요.
    """
    
    response1 = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": example_prompt}],
        temperature=0.7
    )
    
    examples = response1.choices[0].message.content
    
    print("생성된 예시들:")
    print(examples)
    print("\n" + "="*80 + "\n")
    
    # Step 2: 생성된 예시를 활용해 원래 문제 풀이
    final_prompt = f"""
    다음 예시들을 참고하여 문제를 풀어주세요:
    
    {examples}
    
    이제 다음 문제를 같은 방식으로 풀어주세요:
    {question}
    """
    
    response2 = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": final_prompt}],
        temperature=0.3
    )
    
    return response2.choices[0].message.content

# 테스트
question = "한 농장에 닭과 소가 있습니다. 머리는 총 30개이고 다리는 총 84개입니다. 닭과 소는 각각 몇 마리인가요?"

result = auto_cot(question, num_examples=1)
print("최종 답변:")
print(result)

생성된 예시들:
### 문제:
한 농장에 양과 염소가 있습니다. 머리는 총 40개이고 다리는 총 112개입니다. 양과 염소는 각각 몇 마리인가요?

### 단계별 풀이:

1. **변수 설정**:
   - 양의 수를 \( x \)라고 하고, 염소의 수를 \( y \)라고 합시다.

2. **머리 수식 세우기**:
   - 양과 염소는 각각 1개의 머리를 가지고 있습니다. 따라서 머리의 총 수는 다음과 같이 표현할 수 있습니다:
   \[
   x + y = 40 \quad (1)
   \]

3. **다리 수식 세우기**:
   - 양은 4개의 다리를 가지고 있고, 염소도 4개의 다리를 가지고 있습니다. 따라서 다리의 총 수는 다음과 같이 표현할 수 있습니다:
   \[
   4x + 4y = 112 \quad (2)
   \]

4. **식 간소화**:
   - 식 (2)에서 4로 나누어 간소화할 수 있습니다:
   \[
   x + y = 28 \quad (3)
   \]

5. **식 (1)과 식 (3) 동시에 풀기**:
   - 식 (1)과 식 (3)을 비교해보면:
   \[
   x + y = 40 \quad (1)
   \]
   \[
   x + y = 28 \quad (3)
   \]

   두 식은 모순이므로, 실제로는 다리 수식을 다시 풀어보겠습니다.
   - 원래 식 (2)에서 \( 4(x + y) = 112 \) 이므로,
   \[
   x + y = 28
   \]

6. **식 (1)에서 y를 구하기**:
   - 식 (1)에서 \( y \)를 구합니다:
   \[
   y = 40 - x \quad (4)
   \]

7. **(4)식의 값을 (3)식에 대입하기**:
   - (4)의 값을 (3)에 대입하겠습니다:
   \[
   x + (40 - x) = 28
   \]
   이 식을 정리하면:
   \[
   40 = 28
   \]

   다시 계산해보겠습니다.

8. **올바른 다리 수식 확인**:
   - \( 4x + 4y

## 2.5 Self-Consistency

여러 번 추론하여 가장 일관된 답 선택

In [25]:
from collections import Counter

def self_consistency(question, num_samples=5):
    """Self-Consistency 구현"""
    cot_prompt = question + "\n\n단계별로 생각하며 풀어보세요. 최종 답만 명확히 표시하세요."
    
    answers = []
    
    for i in range(num_samples):
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": cot_prompt}],
            temperature=0.7  # 다양성을 위해 temperature 높임
        )
        
        answer = response.choices[0].message.content
        answers.append(answer)
        print(f"\n시도 {i+1}:")
        print(answer)
        print("-" * 80)
    
    # 최종 답변 추출 및 투표
    # 실제로는 더 정교한 파싱이 필요
    print("\n" + "="*80)
    print("\n💡 Self-Consistency: 여러 추론 경로 중 가장 일관된 답을 선택합니다.")
    
    return answers

# 테스트
question = """
한 가게에서 사과 3개를 2000원에 팔고 있습니다.
10개를 사면 10% 할인을 해줍니다.
15개를 사려면 얼마를 내야 하나요?
"""

results = self_consistency(question, num_samples=3)


시도 1:
1. 사과 3개의 가격: 2000원
2. 사과 1개의 가격: 2000원 ÷ 3 = 약 666.67원
3. 사과 15개의 가격: 15개 × 666.67원 = 10000원 (정확히 계산하면 10000원)
4. 10개 이상 구매 시 10% 할인 적용: 10000원 × 0.10 = 1000원 (할인액)
5. 최종 가격: 10000원 - 1000원 = 9000원

최종 금액: **9000원**
--------------------------------------------------------------------------------

시도 2:
1. 사과의 가격: 3개에 2000원이므로 1개 가격은 2000원 ÷ 3 = 약 666.67원입니다.

2. 15개를 사면 기본 가격은 15개 × 666.67원 = 10000.05원입니다. 

3. 그러나 10개 이상 구매 시 10% 할인이 적용됩니다.

4. 기본 가격 10000.05원에 10% 할인: 10000.05원 × 0.10 = 1000.005원입니다.

5. 할인된 가격: 10000.05원 - 1000.005원 = 9000.045원입니다.

6. 최종적으로 소수점 아래를 반올림하면 9000원입니다.

최종 금액: **9000원**.
--------------------------------------------------------------------------------

시도 3:
1. 사과 3개의 가격: 2000원
2. 사과 1개의 가격: 2000원 ÷ 3 = 약 666.67원
3. 사과 15개의 가격: 15 × 666.67원 = 약 10000원
4. 10개 이상 구매 시 10% 할인 적용: 10000원 × 0.10 = 1000원
5. 할인 후 최종 가격: 10000원 - 1000원 = 9000원

최종 가격: 9000원
--------------------------------------------------------------------------------


💡 Self-Consiste

# 3. RAG (Retrieval-Augmented Generation)

외부 문서를 검색하여 답변의 정확도를 높이는 기법

## 3.1 RAG 기본 개념

In [26]:
# 샘플 문서 (회사 규정)
documents = [
    {
        "id": 1,
        "title": "휴가 정책",
        "content": """회사 휴가 정책:
        1. 연차: 입사 1년차 15일, 2년차부터 매년 1일씩 추가 (최대 25일)
        2. 병가: 연간 10일, 의사 소견서 필요
        3. 경조사: 본인 결혼 5일, 직계가족 사망 3일
        4. 휴가 신청: 최소 3일 전 상사 승인 필요
        5. 미사용 연차: 다음 해 이월 불가, 금전 보상 가능"""
    },
    {
        "id": 2,
        "title": "근무 시간",
        "content": """근무 시간 규정:
        1. 정규 근무: 평일 09:00 - 18:00 (주 40시간)
        2. 점심시간: 12:00 - 13:00 (1시간)
        3. 유연근무제: 코어타임 11:00-16:00 준수
        4. 재택근무: 주 2회 가능, 사전 신청 필요
        5. 초과근무: 사전 승인 필요, 시간당 1.5배 수당"""
    },
    {
        "id": 3,
        "title": "복리후생",
        "content": """복리후생 제도:
        1. 건강검진: 연 1회 종합검진 지원
        2. 식대: 월 20만원 식비 지원
        3. 교육비: 연 300만원 한도 업무 관련 교육비 지원
        4. 경조사비: 직원 경조사 시 10-50만원 지원
        5. 자녀 학자금: 중고등학생 자녀 1인당 연 200만원 지원"""
    },
    {
        "id": 4,
        "title": "인사 평가",
        "content": """인사 평가 제도:
        1. 평가 주기: 반기별 (6월, 12월)
        2. 평가 항목: 업무성과 70%, 태도 30%
        3. 등급: S, A, B, C, D (5단계)
        4. 보상: 성과급 차등 지급 (기본급의 0-400%)
        5. 피드백: 평가 후 1:1 면담 진행"""
    },
]

print("📚 문서 데이터베이스:")
for doc in documents:
    print(f"  - {doc['title']}")

📚 문서 데이터베이스:
  - 휴가 정책
  - 근무 시간
  - 복리후생
  - 인사 평가


## 4.2 문서 임베딩

In [27]:
def get_embedding(text, model="text-embedding-3-small"):
    """텍스트를 벡터로 변환"""
    text = text.replace("\n", " ")
    response = client.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

# 모든 문서 임베딩
print("⏳ 문서 임베딩 중...\n")

for doc in documents:
    doc["embedding"] = get_embedding(doc["content"])
    print(f"✅ '{doc['title']}' 임베딩 완료 (차원: {len(doc['embedding'])})")

print("\n✨ 모든 문서 임베딩 완료!")

⏳ 문서 임베딩 중...

✅ '휴가 정책' 임베딩 완료 (차원: 1536)
✅ '근무 시간' 임베딩 완료 (차원: 1536)
✅ '복리후생' 임베딩 완료 (차원: 1536)
✅ '인사 평가' 임베딩 완료 (차원: 1536)

✨ 모든 문서 임베딩 완료!


## 4.3 유사도 검색

In [28]:
def cosine_similarity(vec1, vec2):
    """코사인 유사도 계산"""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def search_documents(query, top_k=2):
    """질문과 가장 유사한 문서 검색"""
    # 질문 임베딩
    query_embedding = get_embedding(query)
    
    # 모든 문서와 유사도 계산
    similarities = []
    for doc in documents:
        similarity = cosine_similarity(query_embedding, doc["embedding"])
        similarities.append((doc, similarity))
    
    # 유사도 순으로 정렬
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # 상위 k개 반환
    return similarities[:top_k]

# 테스트
query = "재택근무를 하려면 어떻게 해야 하나요?"

print(f"🔍 질문: {query}\n")
print("검색 결과:\n")

results = search_documents(query, top_k=2)

for i, (doc, score) in enumerate(results, 1):
    print(f"{i}. {doc['title']} (유사도: {score:.4f})")
    print(f"   내용: {doc['content'][:100]}...\n")

🔍 질문: 재택근무를 하려면 어떻게 해야 하나요?

검색 결과:

1. 근무 시간 (유사도: 0.5100)
   내용: 근무 시간 규정:
        1. 정규 근무: 평일 09:00 - 18:00 (주 40시간)
        2. 점심시간: 12:00 - 13:00 (1시간)
        3...

2. 휴가 정책 (유사도: 0.3082)
   내용: 회사 휴가 정책:
        1. 연차: 입사 1년차 15일, 2년차부터 매년 1일씩 추가 (최대 25일)
        2. 병가: 연간 10일, 의사 소견서 필요
     ...



## 4.4 컨텍스트 기반 답변 생성

In [29]:
def rag_answer(question, top_k=2):
    """RAG 기반 질의응답"""
    
    # 1. 관련 문서 검색
    print(f"📋 질문: {question}\n")
    print("⏳ 관련 문서 검색 중...\n")
    
    relevant_docs = search_documents(question, top_k=top_k)
    
    print("✅ 검색된 문서:")
    for i, (doc, score) in enumerate(relevant_docs, 1):
        print(f"  {i}. {doc['title']} (유사도: {score:.4f})")
    
    # 2. 컨텍스트 구성
    context = "\n\n".join([
        f"[문서 {i}: {doc['title']}]\n{doc['content']}"
        for i, (doc, _) in enumerate(relevant_docs, 1)
    ])
    
    # 3. 프롬프트 구성
    prompt = f"""
다음 문서들을 참고하여 질문에 답변해주세요.
문서에 없는 내용은 추측하지 말고 "문서에서 찾을 수 없습니다"라고 답하세요.

참고 문서:
```
{context}
```

질문: {question}

답변:
"""
    
    # 4. GPT로 답변 생성
    print("\n⏳ 답변 생성 중...\n")
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    
    answer = response.choices[0].message.content
    
    print("="*80)
    print("\n🤖 답변:")
    print(answer)
    print("\n" + "="*80)
    
    # 5. 출처 표시
    print("\n📚 참고 문서:")
    for i, (doc, score) in enumerate(relevant_docs, 1):
        print(f"  [{i}] {doc['title']} (유사도: {score:.4f})")
    
    return answer

# 테스트
rag_answer("재택근무를 하려면 어떻게 해야 하나요?")

📋 질문: 재택근무를 하려면 어떻게 해야 하나요?

⏳ 관련 문서 검색 중...

✅ 검색된 문서:
  1. 근무 시간 (유사도: 0.5101)
  2. 휴가 정책 (유사도: 0.3083)

⏳ 답변 생성 중...


🤖 답변:
재택근무를 하려면 사전 신청이 필요합니다.


📚 참고 문서:
  [1] 근무 시간 (유사도: 0.5101)
  [2] 휴가 정책 (유사도: 0.3083)


'재택근무를 하려면 사전 신청이 필요합니다.'

## 4.5 다양한 질문으로 RAG 테스트

In [30]:
# 여러 질문 테스트
test_questions = [
    "입사 3년차 직원의 연차는 며칠인가요?",
    "인사 평가는 언제 하나요?",
    "식대는 얼마나 지원되나요?",
    "주식 투자에 대한 규정은?",  # 문서에 없는 질문
]

for i, question in enumerate(test_questions, 1):
    print(f"\n\n{'#'*80}")
    print(f"# 테스트 {i}/{len(test_questions)}")
    print(f"{'#'*80}\n")
    
    rag_answer(question, top_k=2)



################################################################################
# 테스트 1/4
################################################################################

📋 질문: 입사 3년차 직원의 연차는 며칠인가요?

⏳ 관련 문서 검색 중...

✅ 검색된 문서:
  1. 휴가 정책 (유사도: 0.5334)
  2. 근무 시간 (유사도: 0.4016)

⏳ 답변 생성 중...


🤖 답변:
입사 3년차 직원의 연차는 16일입니다. (2년차부터 매년 1일씩 추가되므로, 15일 + 1일 = 16일)


📚 참고 문서:
  [1] 휴가 정책 (유사도: 0.5334)
  [2] 근무 시간 (유사도: 0.4016)


################################################################################
# 테스트 2/4
################################################################################

📋 질문: 인사 평가는 언제 하나요?

⏳ 관련 문서 검색 중...

✅ 검색된 문서:
  1. 인사 평가 (유사도: 0.5498)
  2. 휴가 정책 (유사도: 0.2238)

⏳ 답변 생성 중...


🤖 답변:
인사 평가는 반기별로 진행됩니다. (6월, 12월)


📚 참고 문서:
  [1] 인사 평가 (유사도: 0.5498)
  [2] 휴가 정책 (유사도: 0.2238)


################################################################################
# 테스트 3/4
################################################################################

📋 질문: 식대는 얼

## 4.6 RAG 시스템 클래스로 정리

In [31]:
class SimpleRAG:
    """간단한 RAG 시스템"""
    
    def __init__(self, client, model="gpt-4o-mini", embedding_model="text-embedding-3-small"):
        self.client = client
        self.model = model
        self.embedding_model = embedding_model
        self.documents = []
    
    def add_document(self, title, content):
        """문서 추가"""
        embedding = self._get_embedding(content)
        self.documents.append({
            "title": title,
            "content": content,
            "embedding": embedding
        })
    
    def _get_embedding(self, text):
        """텍스트 임베딩"""
        text = text.replace("\n", " ")
        response = self.client.embeddings.create(
            input=[text],
            model=self.embedding_model
        )
        return response.data[0].embedding
    
    def _cosine_similarity(self, vec1, vec2):
        """코사인 유사도"""
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    
    def search(self, query, top_k=2):
        """문서 검색"""
        query_embedding = self._get_embedding(query)
        
        similarities = [
            (doc, self._cosine_similarity(query_embedding, doc["embedding"]))
            for doc in self.documents
        ]
        
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_k]
    
    def answer(self, question, top_k=2, verbose=False):
        """질문에 답변"""
        # 검색
        relevant_docs = self.search(question, top_k=top_k)
        
        if verbose:
            print(f"🔍 검색된 문서:")
            for i, (doc, score) in enumerate(relevant_docs, 1):
                print(f"  {i}. {doc['title']} (유사도: {score:.4f})")
            print()
        
        # 컨텍스트 구성
        context = "\n\n".join([
            f"[문서: {doc['title']}]\n{doc['content']}"
            for doc, _ in relevant_docs
        ])
        
        # 프롬프트
        prompt = f"""
다음 문서들을 참고하여 질문에 답변해주세요.
문서에 없는 내용은 "문서에서 찾을 수 없습니다"라고 답하세요.

참고 문서:
```
{context}
```

질문: {question}
"""
        
        # 답변 생성
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        
        return response.choices[0].message.content

# RAG 시스템 생성 및 사용
print("📚 RAG 시스템 초기화...\n")
rag = SimpleRAG(client)

# 문서 추가
print("📄 문서 추가 중...")
for doc in documents:
    rag.add_document(doc["title"], doc["content"])
    print(f"  ✅ {doc['title']}")

print(f"\n✨ 총 {len(rag.documents)}개 문서 등록 완료!")

# 질문
print("\n" + "="*80 + "\n")
question = "교육비는 얼마나 지원받을 수 있나요?"
print(f"❓ 질문: {question}\n")

answer = rag.answer(question, verbose=True)
print(f"💡 답변: {answer}")

📚 RAG 시스템 초기화...

📄 문서 추가 중...
  ✅ 휴가 정책
  ✅ 근무 시간
  ✅ 복리후생
  ✅ 인사 평가

✨ 총 4개 문서 등록 완료!


❓ 질문: 교육비는 얼마나 지원받을 수 있나요?

🔍 검색된 문서:
  1. 복리후생 (유사도: 0.4638)
  2. 인사 평가 (유사도: 0.2201)

💡 답변: 교육비는 연 300만원 한도로 업무 관련 교육비가 지원됩니다.


## 4.7 RAG vs 일반 질문 비교

In [32]:
question = "우리 회사의 재택근무 정책은 어떻게 되나요?"

print("="*80)
print("❌ RAG 없이 (일반 질문)")
print("="*80 + "\n")

# RAG 없이
response_without_rag = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": question}],
    temperature=0.3
)

print(response_without_rag.choices[0].message.content)

print("\n" + "="*80)
print("✅ RAG 사용")
print("="*80 + "\n")

# RAG 사용
answer_with_rag = rag.answer(question, verbose=True)
print(f"\n{answer_with_rag}")

print("\n" + "="*80)
print("\n💡 차이점:")
print("  - RAG 없이: 일반적인 답변 (회사별로 다를 수 있음)")
print("  - RAG 사용: 회사 문서 기반 정확한 답변")

❌ RAG 없이 (일반 질문)

재택근무 정책은 회사마다 다를 수 있으므로, 정확한 내용을 확인하려면 회사의 인사부서나 내부 문서를 참조하는 것이 가장 좋습니다. 일반적으로 재택근무 정책에는 근무 시간, 의사소통 방법, 업무 성과 평가 기준, 장비 지원, 보안 규정 등이 포함될 수 있습니다. 

혹시 특정한 질문이나 궁금한 점이 있으시면 말씀해 주세요!

✅ RAG 사용

🔍 검색된 문서:
  1. 근무 시간 (유사도: 0.4707)
  2. 휴가 정책 (유사도: 0.3506)


우리 회사의 재택근무 정책은 주 2회 가능하며, 사전 신청이 필요합니다.


💡 차이점:
  - RAG 없이: 일반적인 답변 (회사별로 다를 수 있음)
  - RAG 사용: 회사 문서 기반 정확한 답변


## RAG (w/ Streamlit)

지금까지 배운 내용을 종합하여 Streamlit을 활용한 RAG 시스템을 만들어봅시다!

Streamlit은 쥬피터 노트북에서는 실행이 불가능해서, streamlit_rag.py 형태로 코드를 뒀습니다.
각자 터미널을 열고 아래처럼 실행해보세요:

```
cd ~/metacode-202014/02.ChatGPT-API+
streamlit run rag_demo.py
```